In [17]:
from xgboost.spark import SparkXGBRegressor
spark_reg_estimator = SparkXGBRegressor(
  features_col="features",
  label_col="label",
  num_workers=2,
)

In [18]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("data.csv") \
    .getOrCreate()

# Read the CSV file into a Spark DataFrame
file_path = "/home/ubuntu/data.csv"  # Replace this with the path to your CSV file
train_spark_dataframe = spark.read.csv(file_path, header=True, inferSchema=True)

# Show the schema and the first few rows of the DataFrame
train_spark_dataframe.printSchema()
train_spark_dataframe.show(5)

root
 |-- trip_distance: double (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- pickup_hour: integer (nullable = true)

+--------------------+---------------+------------+------------+--------------------+-----------+
|       trip_distance|passenger_count|PULocationID|DOLocationID|          tip_amount|pickup_hour|
+--------------------+---------------+------------+------------+--------------------+-----------+
| -0.3661467419730907|              1|         151|         239|-0.01412124292201...|          0|
|-0.07707474953506906|              1|         239|         246|-0.03583707603552034|          0|
|  -0.760335822570393|              3|         236|         236|-0.06924605005629836|         13|
|  -0.760335822570393|              5|         193|         193|-0.06924605005629836|         15|
|  -0.760335822570393|              5| 

In [19]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler

# Start Spark session if it's not already started
# Uncomment the next two lines if the Spark session isn't initialized
# spark = SparkSession.builder \
#     .appName("Tip Prediction Model") \
#     .getOrCreate()

# Define the assembler with the input columns that will be features
assembler = VectorAssembler(
    inputCols=["trip_distance", "passenger_count", "PULocationID", "DOLocationID", "pickup_hour"],
    outputCol="features"
)

# Assuming 'train_spark_dataframe' is already loaded and defined
# Transform the data to create feature vectors
transformed_dataframe = assembler.transform(train_spark_dataframe)

# Select only the necessary columns, including the newly created 'features' and the label 'tip_amount'
final_dataframe = transformed_dataframe.select("features", "tip_amount")

# Split the data approximately into 70% training and 30% testing
train_data, test_data = final_dataframe.randomSplit([0.7, 0.3], seed=42)

In [20]:
from xgboost.spark import SparkXGBRegressor

# Create an instance of SparkXGBRegressor
xgb_regressor = SparkXGBRegressor(
    features_col="features",
    label_col="tip_amount",
    prediction_col="predicted_tip_amount",
    objective="reg:squarederror",
    numWorkers=1,
    maxDepth=6,
    eta=0.1
)

In [21]:
xgb_regressor_model = xgb_regressor.fit(train_data)

2024-04-14 16:45:15,015 INFO XGBoost-PySpark: _fit Running xgboost-2.0.3 on 1 workers with
	booster params: {'objective': 'reg:squarederror', 'device': 'cpu', 'numWorkers': 1, 'maxDepth': 6, 'eta': 0.1, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
[16:47:28] task 0 got new rank 0                                    (0 + 1) / 1]
/home/ubuntu/.local/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:48:05] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "maxDepth", "numWorkers" } are not used.

  warnings.warn(smsg, UserWarning)
2024-04-14 16:49:41,185 INFO XGBoost-PySpark: _fit Finished xgboost training!   


In [23]:
from pyspark.ml.evaluation import RegressionEvaluator

# Assuming 'xgb_regressor_model' has already been trained and 'test_data' prepared
predictions = xgb_regressor_model.transform(test_data)

# Evaluate the model using the correct parameter names
evaluator = RegressionEvaluator(
    labelCol="tip_amount",  # Correct parameter name is 'labelCol' not 'label_col'
    predictionCol="predicted_tip_amount",  # Correct parameter name is 'predictionCol'
    metricName="rmse"  # This is correct; other options could be "mae" or "r2"
)

# Calculate RMSE
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

INFO:XGBoost-PySpark:Do the inference on the CPUs                   (0 + 2) / 9]
2024-04-14 16:50:38,373 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
INFO:XGBoost-PySpark:Do the inference on the CPUs                   (2 + 2) / 9]
2024-04-14 16:51:09,568 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
INFO:XGBoost-PySpark:Do the inference on the CPUs                   (4 + 2) / 9]
2024-04-14 16:51:41,912 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2024-04-14 16:52:13,505 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
INFO:XGBoost-PySpark:Do the inference on the CPUs
2024-04-14 16:52:45,396 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs


Root Mean Squared Error (RMSE) on test data = 1.82001
